In [9]:
!pip install opencv-python tensorflow

  Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [8]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

# Załaduj model
model = tf.keras.models.load_model('models/lstm_sign_model.keras')

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
import cv2
import mediapipe as mp
import tensorflow as tf
import numpy as np
import pickle
from collections import deque

# Mapowanie etykiet niestandardowych na polskie litery
label_map = {
    "A": "A", "A+": "Ą", "B": "B", "C": "C", "C+": "Ć",
    "D": "D", "E": "E", "E+": "Ę", "F": "F", "G": "G",
    "H": "H", "I": "I", "J": "J", "K": "K", "L": "L", "L+": "Ł",
    "M": "M", "N": "N", "N+": "Ń", "O": "O", "O+": "Ó",
    "P": "P", "Q": "Q", "R": "R", "S": "S", "S+": "Ś",
    "T": "T", "U": "U", "V": "V", "W": "W", "X": "X",
    "Y": "Y", "Z": "Z", "Z+": "Ż", "Z++": "Ź"
}

# Wczytaj LabelEncoder
with open('label_encoder.pkl', 'rb') as f:
    le = pickle.load(f)

# Inicjalizacja MediaPipe
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)

# Wczytaj model .keras
model = tf.keras.models.load_model('models/lstm_sign_model.keras')

# Kolejka do przechowywania 3 ostatnich klatek z landmarkami
sequence = deque(maxlen=3)

# Kamera
cap = cv2.VideoCapture(0)
print("\nNacisnij 'q' aby zakonczyc.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Nie mozna odczytac klatki z kamery.")
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    prediction_text = ""

    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        landmarks = []
        for lm in hand_landmarks.landmark:
            landmarks.extend([lm.x, lm.y, lm.z])

        sequence.append(landmarks)

        if len(sequence) == 3:
            input_seq = np.array(sequence).reshape(1, 3, 63)
            prediction = model.predict(input_seq, verbose=0)
            predicted_index = prediction.argmax()
            predicted_class = le.inverse_transform([predicted_index])[0]
            predicted_label = label_map.get(predicted_class, predicted_class)
            prediction_text = f'Predykcja: {predicted_label}'

    if prediction_text:
        cv2.putText(frame, prediction_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow("Kamera", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("Zakonczenie.")


I0000 00:00:1748498355.170581    6245 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1748498355.173054   28391 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.6), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)
W0000 00:00:1748498355.209799   28382 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748498355.237981   28383 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.



Nacisnij 'q' aby zakonczyc.
Zakonczenie.
